# 三者对比
![](../images/Python并发编程/三者对比.png)

# 三者选择
![](../images/Python并发编程/三者选择.png)

# 全局解释器锁
## Python速度慢的两大原因
1. Python是动态类型语言，边解释边执行
2. GIL无法利用多核CPU并发执行
## GIL是什么
全局解释器锁是编程语言解释器用于同步线程的一种机制，它使得任何时刻只有一个线程在执行

即便在多核心处理器上，使用GIL的解释器也只允许同一时间执行一个线程
![](../images/Python并发编程/GIL.png)

## 为什么会有GIL？
在一个古老的单核CPU上调度多个线程任务，大家相互共享一个全局锁，谁在CPU执行，谁就占有这把锁，直到这个线程因为IO操作或者Timer Tick到期让出CPU，没有在执行的线程就安静的等待着这把锁。使用全局解释器锁的方案是最简单的

由于多个线程同时对数据进行操作，会引发数据不一致，导致内存泄漏，我们可以对其进行加锁，所以Cpython就创建了GIL锁
但是既然有了锁，一个对象就需要一把锁，那么多个对象就会有多把锁，可能会给我们带来2个问题

1. 死锁（线程之间互相争抢锁的资源）
2. 反复获取和释放锁而导致性能降低

为了保证单线程情况下python的正常执行和效率，GIL锁（单一锁）由此产生了，它添加了一个规则，即任何Python字节码的执行都需要获取解释器锁。这样可以防止死锁（因为只有一个锁），并且不会带来太多的性能开销。但这实际上使所有受CPU约束的Python程序（指的是CPU密集型程序）都是单线程的。

### 另一个思路：为什么 JVM 不需要 GIL？
不妨思考这样一个问题：为什么 JVM 不需要 GIL ？试想一下，你费尽九牛二虎之力写了一个 Hash Map，准备丢给别人用，突然别人告诉你我会多线程同时访问你这个东西，此时你有两个选择：

改造底层设计，例如给哈希空间中的每一个桶都加上锁，让整个 Hash Map 能在大部分情境下真正支持多线程并发；当然如果你这个 Map 带缩扩容之类的，那要改造成线程安全的就更麻烦了。

直接给整个 Map 加一个全局锁，谁想访问就得先拿这个锁，一分钟改完。这里的 Map 就是 CPython，全局锁就是 GIL，只是复杂程度要高很多。

CPython 在两种策略中选择了后者（这是 “CPython 的内存管理为什么不是线程安全的“ 的回答，选择如此），因此同一时刻永远只有一个线程在真正执行；而 JVM 在两种策略中选择了前者，使得它真正支持多核并发。

但是后者并非除了省事之外就一无是处。选择后者，CPython 运行起来像是这样的：

```
取得GIL(); 
...
...
...
释放GIL();
```

选择前者，CPython 运行起来就像这样：

```
取得某个锁();
...
释放某个锁();
...
...
取得某个锁();
...
释放某个锁();
```

所以，对于解释型语言，支持线程多核并行是有代价的，就是单个线程效率的下降。一旦底层解释器采用了前者那样的设计，用更多小粒度的锁来替代一个大锁，那么即便是单线程程序，也会不可避免地在运行过程中有许多取得和释放锁的操作，会让效率大打折扣。

线程效率不仅仅对单线程程序重要，比如，一些 IO 密集型的程序，看起来有很多线程，但大部分线程大多数时间都在挂机，实际疯狂干活的可能就一两个线程，这种场景下，单位线程效率也是非常重要的，即便用上 2 个核一起跑两个线程，单位线程效率下降多了，也未必快得了多少。

总结一下，主要是出于历史原因和单位线程效率的考虑， CPython 至今还有 GIL

## 有GIL一定线程安全吗？
有GIL并不意味着python一定是线程安全的，那什么时候安全，什么时候不安全，我们必须搞清楚。之前我们已经说过，一个线程有两种情况下会释放全局解释器锁，一种情况是在该线程进入IO操作之前，会主动释放GIL。另一种情况是解释器不间断运行了1000字节码（Py2）或运行15毫秒（Py3）后，该线程也会放弃GIL。既然一个线程可能随时会失去GIL，那么这就一定会涉及到线程安全的问题。GIL虽然从设计的出发点就是考虑到线程安全，但这种线程安全是粗粒度的线程安全，即不需要程序员自己对线程进行加锁处理（同理，所谓细粒度就是指程序员需要自行加、解锁来保证线程安全，典型代表是 Java , 而 CPthon 中是粗粒度的锁，即语言层面本身维护着一个全局的锁机制,用来保证线程安全）。那么什么时候需要加锁，什么时候不需要加锁，这个需要具体情况具体分析。下面我们就来针对每种可能的情况进行分析和总结。

首先来看第一种线程释放GIL的情况。假设现在线程A因为进入IO操作而主动释放了GIL，那么在这种情况下，由于线程A的IO操作等待时间不确定，那么等待的线程B一定会得到GIL锁，这种比较“礼貌的”情况我们一般称为“协同式多任务处理”，相当于大家按照协商好的规则来，线程是安全的，不需要额外加锁。

接下来，我们来看另外一种情况，即线程A是因为解释器不间断执行了1000字节码的指令或不间断运行了15毫秒而放弃了GIL，那么此时实际上线程A和线程B将同时竞争GIL锁。在同时竞争的情况下，实际上谁会竞争成功是不确定的一个结果，所以一般被称为“抢占式多任务处理”，这种情况下当然就看谁抢得厉害了。当然，在python3上由于对GIL做了优化，并且会动态调整线程的优先级，所以线程B的优先级会比较高，但仍然无法肯定线程B就一定会拿到GIL。那么在这种情况下，线程可能就会出现不安全的状态。针对这种纯计算的操作，我们用一段代码来演示下这种线程不安全的状态。代码如下：


In [1]:
import threading

n = 0

def add():
    global n
    for i in range(1000000):
        n = n + 1
def sub():
    global n
    for i in range(1000000):
        n = n - 1

if __name__ == "__main__":
    t1 = threading.Thread(target=add)
    t2 = threading.Thread(target=sub)

    t1.start()
    t2.start()

    t1.join()
    t2.join()

    print("n的值为:", n)


n的值为: 65763


上面的代码很简单，分别用线程1和线程2对全局变量n进行了1000000次的加和减操作。如果线程安全的话，那么最终的结果n应该还是为0。但实际上，我们运行之后，会发现这个n的值有时大有时小，完全不确定。这就是典型的多个线程操作同一个全局变量造成的线程不安全的问题。

在线程中，我们主要是执行了一个加法和减法的操作。为了方便说明问题，我们把函数最简化到一个加法函数和一个减法函数，来分析它们的字节码执行过程，来看看释放GIL锁是怎么引起这个问题的。演示代码如下：

In [2]:
import dis
n = 0

def add():
    global n
    n = n + 1

print(dis.dis(add))

def sub():
    global n
    n = n - 1
print(dis.dis(sub))

  6           0 LOAD_GLOBAL              0 (n)
              2 LOAD_CONST               1 (1)
              4 BINARY_ADD
              6 STORE_GLOBAL             0 (n)
              8 LOAD_CONST               0 (None)
             10 RETURN_VALUE
None
 12           0 LOAD_GLOBAL              0 (n)
              2 LOAD_CONST               1 (1)
              4 BINARY_SUBTRACT
              6 STORE_GLOBAL             0 (n)
              8 LOAD_CONST               0 (None)
             10 RETURN_VALUE
None


dis模块中的dis方法可以打印出一个函数对应的字节码执行过程，所以非常方便我们进行分析。运行结果如上：

不管是加法还是减法运算，都会分为4步完成。以加法为例，第一步是LOAD_GLOBAL(加载全局变量n)，第二步LOAD_CONST(加载常量1)，第三步进行二进制的加法，第四步将计算结果存储到全局变量n中，加法计算结束。这四个指令如果能够保证被作为一个整体完整地运行，那么是不会产生问题的，但根据前面说的线程释放GIL的原则，那么很有可能在线程正在执行这四步中的任何一步的时候释放掉GIL而进入等待状态，这个时候发生的事情就比较有意思了。为了方便大家理解，我拿一种比较极端的情况来说明一下。比如我们在加法运算中，正准备执行第四步的时候，很不幸失去了GIL，进入等待状态（注意此时n值仍然为0）。减法运算的线程开始执行，它加载了全局变量n（值为0），并进行减法相关的计算，它也在执行第三步的时候失去了GIL，此时它进入等待状态，加法运算继续。上一次加法计算继续运行第4步，即把加法运算结果赋值给全局变量n，那么此时n的值为1。同样道理，减法操作拿回GIL时，它之前已经加载了为0的n的值，所以它继续操作到最后赋值那步时，n的值就为0-1=-1。换句话说，n的值要么为1，要么为-1，但我们期望的应该是0。这就造成了线程不安全的情形。最终，经过百万次这样不确定的加减操作，那么结果一定是不确定的。这就是引起这个问题的过程和原因。

接下来，我们还要解决另外一个问题，也就是既然GIL从粗粒度情况下存在线程不安全的可能性，那么是不是所有非IO操作引起的GIL释放都要加锁来解决线程安全的问题。这个问题同样要分情况，因为python跟其他线程自由的语言比如 Java相比，它有很多操作是原子级的，针对原子级的操作，由于方法本身是单个字节码，所以线程没有办法在调用期间放弃GIL。典型的例子比如sort方法，我们同样可以看看这种原子级的操作在python的字节码中是什么样子，代码演示如下：

In [3]:
import dis

lst = [4, 1, 3, 2]

def foo():
    lst.sort()

print(dis.dis(foo))

  6           0 LOAD_GLOBAL              0 (lst)
              2 LOAD_METHOD              1 (sort)
              4 CALL_METHOD              0
              6 POP_TOP
              8 LOAD_CONST               0 (None)
             10 RETURN_VALUE
None


从字节码的角度，调用sort操作是原子级无法再分的，所以线程不会在执行期间发生GIL释放的情况，也就是说我们可以认为sort操作是线程安全的，不需要加锁。而我们上面演示的加法和减法操作则不是原子级的，所以我们必须要加锁才能保证线程安全。

所以，总结一下，如果多线程的操作中不是IO密集型，并且计算操作不是原子级的操作时，那么我们需要考虑线程安全问题，否则都不需要考虑线程安全。当然，为了避免担心哪个操作是原子的，我们可以遵循一个简单的原则：始终围绕共享可变状态的读取和写入加锁。毕竟，在 Python 中获取一个 threading.Lock 也就是一行代码的事。

## 怎样规避GIL带来的限制
![](../images/Python并发编程/规避GIL.png)

## Python创建多线程的方法

In [4]:
# 1. 准备一个函数
def my_func(a,b):
    do_craw(a,b)    # 执行爬虫方法
# 2. 怎样创建一个线程
import threading
t = threading.Thread(target=my_func,args=(100,200))
# 3. 启动线程
t.start()
# 4. 等待结束
t.join()

Exception in thread Thread-45:
Traceback (most recent call last):
  File "/Users/cenzhedong/opt/anaconda3/lib/python3.9/threading.py", line 973, in _bootstrap_inner
    self.run()
  File "/Users/cenzhedong/opt/anaconda3/lib/python3.9/threading.py", line 910, in run
    self._target(*self._args, **self._kwargs)
  File "/var/folders/8j/9tptyt0x6cg58rglpp99p35w0000gn/T/ipykernel_88748/12809258.py", line 3, in my_func
NameError: name 'do_craw' is not defined


In [1]:
import requests
urls = [
    f'https://www.cnblogs.com/#p{page}' for page in range(1,50+1)
]
def craw(url):  # 爬虫代码
    r=requests.get(url)
    print(url,len(r.text))
craw(urls[0])

https://www.cnblogs.com/#p1 72647


In [4]:
import threading

def single_thread():    # 单线程爬虫
    print('single thread begin')
    for url in urls:
        craw(url)
    print('single thread end')
def multi_thread():   # 多线程爬虫
    print('multi thread begin')
    threads = []
    for url in urls:
        threads.append(
            threading.Thread(target=craw,args=(url,))
        )
    for thread in threads:
        thread.start()
    for thread in threads:
        thread.join()
    print('multi thread end')

In [7]:
import time
start = time.time()
single_thread()
end = time.time()
print('single thread cost:',end-start,"seconds")


single thread begin
https://www.cnblogs.com/#p1 72645
https://www.cnblogs.com/#p2 72645
https://www.cnblogs.com/#p3 72645
https://www.cnblogs.com/#p4 72645
https://www.cnblogs.com/#p5 72645
https://www.cnblogs.com/#p6 72645
https://www.cnblogs.com/#p7 72645
https://www.cnblogs.com/#p8 72645
https://www.cnblogs.com/#p9 72645
https://www.cnblogs.com/#p10 72645
https://www.cnblogs.com/#p11 72645
https://www.cnblogs.com/#p12 72645
https://www.cnblogs.com/#p13 72645
https://www.cnblogs.com/#p14 72645
https://www.cnblogs.com/#p15 72645
https://www.cnblogs.com/#p16 72645
https://www.cnblogs.com/#p17 72645
https://www.cnblogs.com/#p18 72645
https://www.cnblogs.com/#p19 72645
https://www.cnblogs.com/#p20 72645
https://www.cnblogs.com/#p21 72645
https://www.cnblogs.com/#p22 72645
https://www.cnblogs.com/#p23 72645
https://www.cnblogs.com/#p24 72645
https://www.cnblogs.com/#p25 72645
https://www.cnblogs.com/#p26 72645
https://www.cnblogs.com/#p27 72645
https://www.cnblogs.com/#p28 72645
https://w

In [5]:
start = time.time()
multi_thread()
end = time.time()
print('multi thread cost:',end-start,"seconds")

multi thread begin
https://www.cnblogs.com/#p5 72647
https://www.cnblogs.com/#p2 72647
https://www.cnblogs.com/#p1 72647
https://www.cnblogs.com/#p4 72647
https://www.cnblogs.com/#p27 72647
https://www.cnblogs.com/#p41https://www.cnblogs.com/#p47 72647
 72647
https://www.cnblogs.com/#p31https://www.cnblogs.com/#p44https://www.cnblogs.com/#p18 72647
https://www.cnblogs.com/#p16https://www.cnblogs.com/#p36 72647
https://www.cnblogs.com/#p22 https://www.cnblogs.com/#p40 72647  https://www.cnblogs.com/#p48  72647
https://www.cnblogs.com/#p45 
726477264772647
7264772647

https://www.cnblogs.com/#p42 72647

https://www.cnblogs.com/#p29 72647
https://www.cnblogs.com/#p26
https://www.cnblogs.com/#p23 72647
 https://www.cnblogs.com/#p50 72647
72647https://www.cnblogs.com/#p12
https://www.cnblogs.com/#p21 72647
 72647
https://www.cnblogs.com/#p34https://www.cnblogs.com/#p39 72647
https://www.cnblogs.com/#p38 72647
https://www.cnblogs.com/#p15 72647
https://www.cnblogs.com/#p10https://www.cnblogs

## 多组件的Pipeline技术架构
复杂的事情一般不会一下子做完，而是会分很多中间步骤一步步完成
## 生产者消费者爬虫的架构
![](../images/Python并发编程/爬虫架构.png)

## 多线程数据通信的queue.Queue
可以用于多线程之间的，线程安全（并发同时访问数据不会出现数据冲突）的数据通信

1. 导入类库
import queue
2. 创建Queue
q = queue.Queue()
3. 添加元素(阻塞的)
q.put(item)
4. 获取元素(阻塞的)
item = q.get()
5. 查询状态
    * 查看元素的多少
    q.qsize()
    * 判断是否为空
    q.empty()
    * 判断是否已满
    q.full()

In [9]:
from bs4 import BeautifulSoup
def craw(url):
    r=requests.get(url)
    return r.text

def parse(html):
    # class="post-item-title"
    soup = BeautifulSoup(html,"html.parser")
    links = soup.find_all("a",class_="post-item-title")
    return [(link["href"],link.get_text()) for link in links]

parse(craw(urls[1]))

[('https://www.cnblogs.com/shudaoshan/p/18151059',
  '鸿蒙HarmonyOS实战-ArkUI动画（弹簧曲线动画）'),
 ('https://www.cnblogs.com/FatTiger4399/p/18146367', '低开开发笔记(四)：实现编辑器内拖拽'),
 ('https://www.cnblogs.com/ITnoteforlsy/p/18148656',
  'Qt 从 QTransform 逆向解出 Translate/Scale/Rotate（平移/缩放/旋转）分析'),
 ('https://www.cnblogs.com/isharetech/p/18150815',
  '深度解读《深度探索C++对象模型》之数据成员的存取效率分析（三）'),
 ('https://www.cnblogs.com/Yesi/p/18150797', '使用C# 创建、填写、删除PDF表单域'),
 ('https://www.cnblogs.com/mingupupu/p/18150708', 'WPF/C#：如何显示具有层级关系的数据'),
 ('https://www.cnblogs.com/xiaoqi/p/18150639/golang-sparse-retrival',
  '浅谈sparse vec检索工程化实现'),
 ('https://www.cnblogs.com/guzb/p/18108245/difference-of-thread-pool-implementation',
  '线程池的运行逻辑与你想象的不一样，它是池族中的异类'),
 ('https://www.cnblogs.com/fkxxgis/p/18150594', 'ArcPy自动绘制大量地图并设置地图要素：Python'),
 ('https://www.cnblogs.com/crossoverJie/p/18150585',
  '日志架构演进：从集中式到分布式的Kubernetes日志策略'),
 ('https://www.cnblogs.com/VincentLee/p/18150445',
  'LORS：腾讯提出低秩残差结构，瘦身模型不掉点 | CVPR 2024'),
 ('https://

## 生产者消费者版本的爬虫

In [10]:
import queue
import random
def do_craw(url_queue:queue.Queue,html_queue:queue.Queue):
    while True:
        url = url_queue.get()
        html = craw(url)
        html_queue.put(html)
        print(threading.current_thread().name,f"craw {url}","url_queue.size=",url_queue.qsize())
        time.sleep(random.randint(1,2))

def do_parse(html_queue: queue.Queue,fout):
    while True:
        html = html_queue.get()
        results = parse(html)
        for result in results:
            fout.write(str(result)+'\n')
        print(threading.current_thread().name,f"result.size",len(results),"html_queue.size=",html_queue.qsize())
        time.sleep(random.randint(1,2))

# url_queue = queue.Queue()
# html_queue = queue.Queue()
# for url in urls:
#     url_queue.put(url)
# for idx in range(3):    # 三个线程下载网页
#     t=threading.Thread(target=do_craw, args=(url_queue,html_queue),name=f"craw{idx}")
#     t.start()

# fout = open('02.data.txt','w')
# for idx in range(2):    # 两个线程解析网页并存储
#     t=threading.Thread(target=do_parse, args=(html_queue,fout),name=f"parse{idx}")
#     t.start()


## 线程安全概念介绍
线程安全指的是某个函数、函数库在多线程环境中被调用时，能够正确的处理多个线程之间的共享变量，使得程序功能正确完成

由于线程的执行随时会发生切换，就造成了不可预料的结果，出现线程不安全

In [11]:
class Account:
    def __init__(self,balance):
        self.balance = balance
def draw(account, amount):
    if account.balance>=amount:
        print(threading.current_thread().name,"取钱成功")
        account.balance -= amount
        print(threading.current_thread().name,"余额",account.balance)
    else:
        print(threading.current_thread().name,"取钱失败，余额不足")
account = Account(1000)
ta = threading.Thread(name='ta',target=draw,args=(account,800))
tb = threading.Thread(name='tb',target=draw,args=(account,800))

ta.start()
tb.start()

ta 取钱成功
ta 余额 200
tb 取钱失败，余额不足


In [12]:
class Account:
    def __init__(self,balance):
        self.balance = balance
def draw(account, amount):
    if account.balance>=amount:
        time.sleep(0.1) # 加上这句后必然出问题
        print(threading.current_thread().name,"取钱成功")
        account.balance -= amount
        print(threading.current_thread().name,"余额",account.balance)
    else:
        print(threading.current_thread().name,"取钱失败，余额不足")
account = Account(1000)
ta = threading.Thread(name='ta',target=draw,args=(account,800))
tb = threading.Thread(name='tb',target=draw,args=(account,800))

ta.start()
tb.start()

## 修改线程不安全的代码

In [13]:
lock = threading.Lock()
class Account:
    def __init__(self,balance):
        self.balance = balance
def draw(account, amount):
    with lock:
        if account.balance>=amount:
            time.sleep(1) # 使用lock后，第二个线程无法进入，这里会有1秒啥也不干
            print(threading.current_thread().name,"取钱成功")
            account.balance -= amount
            print(threading.current_thread().name,"余额",account.balance)
        else:
            print(threading.current_thread().name,"取钱失败，余额不足")
account = Account(1000)
ta = threading.Thread(name='ta',target=draw,args=(account,800))
tb = threading.Thread(name='tb',target=draw,args=(account,800))

ta.start()
tb.start()

# 线程池ThreadPoolExecutor
## 线程池的原理
新建线程系统需要分配资源，终止线程系统需要回收资源，如果可以重用线程，则可以减去新建/终止的开销
![](../images/Python并发编程/线程池.png)
## 使用线程池的好处
1. 提升性能：因为建去了大量新建终止线程的开销，重用了线程资源
2. 使用场景：适合处理突发性大量请求或需要大量线程完成任务，但实际任务处理时间较短
3. 防御功能：能有效避免系统因为创建线程过多，而导致系统负荷过大响应变慢等问题
4. 代码优势：比手动新建线程更加简洁优雅
## 线程池语法
![](../images/Python并发编程/线程池语法.png)

In [14]:
import concurrent.futures

# craw
with concurrent.futures.ThreadPoolExecutor() as pool:
    htmls=pool.map(craw,urls)
    htmls = list(zip(urls,htmls))
    for url,html in htmls:
        print(url,len(html))

print("craw over")


tb 取钱成功
tb 余额 200
ta 取钱成功
ta 余额 -600
https://www.cnblogs.com/#p1 72647
https://www.cnblogs.com/#p2 72647
https://www.cnblogs.com/#p3 72647
https://www.cnblogs.com/#p4 72647
https://www.cnblogs.com/#p5 72647
https://www.cnblogs.com/#p6 72647
https://www.cnblogs.com/#p7 72647
https://www.cnblogs.com/#p8 72647
https://www.cnblogs.com/#p9 72647
https://www.cnblogs.com/#p10 72647
https://www.cnblogs.com/#p11 72647
https://www.cnblogs.com/#p12 72647
https://www.cnblogs.com/#p13 72647
https://www.cnblogs.com/#p14 72647
https://www.cnblogs.com/#p15 72647
https://www.cnblogs.com/#p16 72647
https://www.cnblogs.com/#p17 72647
https://www.cnblogs.com/#p18 72647
https://www.cnblogs.com/#p19 72647
https://www.cnblogs.com/#p20 72647
https://www.cnblogs.com/#p21 72647
https://www.cnblogs.com/#p22 72647
https://www.cnblogs.com/#p23 72647
https://www.cnblogs.com/#p24 72647
https://www.cnblogs.com/#p25 72647
https://www.cnblogs.com/#p26 72647
https://www.cnblogs.com/#p27 72647
https://www.cnblogs.com/#p2

In [15]:
# parse
# 用submit
with concurrent.futures.ThreadPoolExecutor() as pool:
    futures = {}
    for url, html in htmls:
        future = pool.submit(parse,html)
        futures[future] = url
    
    # for future,url in futures.items():  # 按传入顺序获取返回结果
    #     print(url, future.result())
    for future in concurrent.futures.as_completed(futures):   # 先完成的任务先返回结果
        url = futures[future]
        print(url,future.result())

ta https://www.cnblogs.com/#p24取钱成功 
[('https://www.cnblogs.com/shudaoshan/p/18151059', '鸿蒙HarmonyOS实战-ArkUI动画（弹簧曲线动画）'), ('https://www.cnblogs.com/FatTiger4399/p/18146367', '低开开发笔记(四)：实现编辑器内拖拽'), ('https://www.cnblogs.com/ITnoteforlsy/p/18148656', 'Qt 从 QTransform 逆向解出 Translate/Scale/Rotate（平移/缩放/旋转）分析'), ('https://www.cnblogs.com/isharetech/p/18150815', '深度解读《深度探索C++对象模型》之数据成员的存取效率分析（三）'), ('https://www.cnblogs.com/Yesi/p/18150797', '使用C# 创建、填写、删除PDF表单域'), ('https://www.cnblogs.com/mingupupu/p/18150708', 'WPF/C#：如何显示具有层级关系的数据'), ('https://www.cnblogs.com/xiaoqi/p/18150639/golang-sparse-retrival', '浅谈sparse vec检索工程化实现'), ('https://www.cnblogs.com/guzb/p/18108245/difference-of-thread-pool-implementation', '线程池的运行逻辑与你想象的不一样，它是池族中的异类'), ('https://www.cnblogs.com/fkxxgis/p/18150594', 'ArcPy自动绘制大量地图并设置地图要素：Python'), ('https://www.cnblogs.com/crossoverJie/p/18150585', '日志架构演进：从集中式到分布式的Kubernetes日志策略'), ('https://www.cnblogs.com/VincentLee/p/18150445', 'LORS：腾讯提出低秩残差结构，瘦身模型不掉点 | CVPR 2024')

# Web服务中使用线程池加速
1. 方便地将IO调用并发执行
2. 线程池的线程数目不会无限创建（导致系统挂掉），具有防御功能

# 多进程
## 有了多线程，为什么还要多进程
如果遇到CPU密集型计算，多线程反而降低执行速度
![](../images/Python并发编程/多进程.png)
## 多进程对比多线程
![](../images/Python并发编程/多进程对比多线程.png)

In [16]:
import math,time
from concurrent.futures import ThreadPoolExecutor, ProcessPoolExecutor
PRIMES = [112272535095293]*100
def is_prime(n):
    if n<2:
        return False
    if n==2:
        return True
    if n%2 ==0:
        return False
    sqrt_n = int(math.floor(math.sqrt(n)))
    for i in range(3,sqrt_n+1,2):
        if n%i==0:
            return False
    return True

def single_thread():
    for number in PRIMES:
        is_prime(number)

def multi_thread():
    with ThreadPoolExecutor() as pool:
        pool.map(is_prime,PRIMES)

def multi_process():
    with ProcessPoolExecutor() as pool:
        pool.map(is_prime,PRIMES)

In [17]:
if __name__ == "__main__":
    start = time.time()
    single_thread()
    end = time.time()
    print('single thread, cost:',end-start,'seconds')

    start = time.time()
    multi_thread()
    end = time.time()
    print('multi thread, cost:',end-start,'seconds')

    start = time.time()
    multi_process()
    end = time.time()
    print('multi process, cost:',end-start,'seconds')

single thread, cost: 26.073679208755493 seconds
multi thread, cost: 25.577187061309814 seconds
multi process, cost: 0.1091609001159668 seconds


Process SpawnProcess-1:
Process SpawnProcess-2:
Process SpawnProcess-4:
Process SpawnProcess-3:
Process SpawnProcess-5:
Traceback (most recent call last):
Traceback (most recent call last):
  File "/Users/cenzhedong/opt/anaconda3/lib/python3.9/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/Users/cenzhedong/opt/anaconda3/lib/python3.9/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/cenzhedong/opt/anaconda3/lib/python3.9/concurrent/futures/process.py", line 237, in _process_worker
    call_item = call_queue.get(block=True)
  File "/Users/cenzhedong/opt/anaconda3/lib/python3.9/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/Users/cenzhedong/opt/anaconda3/lib/python3.9/multiprocessing/queues.py", line 122, in get
    return _ForkingPickler.loads(res)
  File "/Users/cenzhedong/opt/anaconda3/lib/python3.9/multiprocessing/process.py", line 108, in run
    self._target(*self._a

# 协程并发
![](../images/Python并发编程/协程并发.png)

## Python异步IO库：asyncio
要用在异步编程中，依赖的库必须支持异步IO特性

爬虫引用中：
requests不支持异步
需要用aiohttp

In [18]:
import asyncio
# 获取事件循环
loop = asyncio.get_event_loop()
# 定义协程
async def myfunc(url):
    await get_url(url)

# 创建task列表
tasks = [loop.create_task(myfunc(url)) for url in urls]
# 执行爬虫事件列表
loop.run_until_complete(asyncio.wait(tasks))

RuntimeError: This event loop is already running

In [3]:
import asyncio
import aiohttp

async def async_craw(url):
    async with aiohttp.ClientSession() as session:
        async with session.get(url) as resp:
            result = await resp.text()
            print(f"craw url: {url},{len(result)}")
loop = asyncio.get_event_loop()
tasks = [
    loop.create_task(async_craw(url)) for url in urls
]

import time
start = time.time()
loop.run_until_complete(asyncio.wait(tasks))
end = time.time()
print("use time seconds: ",end-start)

RuntimeError: This event loop is already running

craw url: https://www.cnblogs.com/#p5,72647
craw url: https://www.cnblogs.com/#p3,72647
craw url: https://www.cnblogs.com/#p10,72647
craw url: https://www.cnblogs.com/#p7,72647
craw url: https://www.cnblogs.com/#p4,72647
craw url: https://www.cnblogs.com/#p9,72647
craw url: https://www.cnblogs.com/#p17,72647
craw url: https://www.cnblogs.com/#p1,72647
craw url: https://www.cnblogs.com/#p23,72647
craw url: https://www.cnblogs.com/#p16,72647
craw url: https://www.cnblogs.com/#p19,72647
craw url: https://www.cnblogs.com/#p8,72647
craw url: https://www.cnblogs.com/#p24,72647
craw url: https://www.cnblogs.com/#p29,72647
craw url: https://www.cnblogs.com/#p13,72647
craw url: https://www.cnblogs.com/#p26,72647
craw url: https://www.cnblogs.com/#p12,72647
craw url: https://www.cnblogs.com/#p2,72647
craw url: https://www.cnblogs.com/#p6,72647
craw url: https://www.cnblogs.com/#p20,72647
craw url: https://www.cnblogs.com/#p22,72647
craw url: https://www.cnblogs.com/#p14,72647
craw url: https://w